### 测试手动调用 flush() 

flush()调用 API:

1. https://milvus.io/api-reference/pymilvus/v2.5.x/ORM/Collection/flush.md

2. https://docs.zilliz.com.cn/reference/python/python/Management-flush

In [ ]:
# 定义嵌入模型
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embed_model = OpenAIEmbeddings(
    api_key=os.getenv("BL_API_KEY"),
    base_url=os.getenv("BL_BASE_URL"),
    model="text-embedding-v3",
    dimensions=1024,
    check_embedding_ctx_length=False
)

In [ ]:
# 加载向量数据库
from langchain_milvus import Milvus, BM25BuiltInFunction

dense_vs = Milvus(
    collection_name='dense_hotpotqa_500',
    embedding_function=embed_model
)

hybrid_vs = Milvus(
    collection_name='hybrid_hotpotqa_500',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)

In [ ]:
dense_vs.client.flush(collection_name='dense_hotpotqa_500')
hybrid_vs.client.flush(collection_name='hybrid_hotpotqa_500')

In [ ]:
import json
import pprint
from loguru import logger as log

# 把所有问题的 embedding 存到文件里，节省不必要的 token 浪费

# 加载问题
all_questions = []
question_path = '../../evaluation/hotpotqa/hybrid_test_data_500.json'
with open(question_path, 'r', encoding='utf-8') as f:
    all_questions = json.load(f)

log.info(f'加载了 {len(all_questions)} 个问题')
pprint.pprint(all_questions[0])

2025-07-25 09:36:59.425 | INFO     | __main__:<module>:13 - 加载了 500 个问题


{'_id': '5a8ba761554299240d9c2066',
 'answer': 'nearly 80 years',
 'context': [['Bus Stop (TV series)',
              ['Bus Stop is a 26-episode American drama which aired on ABC '
               'from October 1, 1961, until March 25, 1962, starring Marilyn '
               'Maxwell as Grace Sherwood, the owner of a bus station and '
               'diner in the fictitious town of Sunrise in the Colorado '
               'Rockies.',
               ' The program was adapted from William Inge\'s play, "Bus '
               'Stop", and Inge was a script consultant for the series, which '
               'followed the lives of travelers passing through the bus '
               'station and the diner.',
               " Maxwell's co-stars were Richard Anderson as District Attorney "
               'Glenn Wagner, Rhodes Reason as Sheriff Will Mayberry, Joan '
               'Freeman as waitress Elma Gahrigner, Bernard Kates as Ralph the '
               'coroner, and Buddy Ebsen as Virge Bles

In [ ]:
# 把问题的 embedding 存入文件
question_embedding_path = '../../evaluation/hotpotqa/hybrid_test_data_w_embedding_500.json'

for question in all_questions:
    embedding = embed_model.embed_query(question['question'])
    question['embedding'] = embedding

with open(question_embedding_path, 'w', encoding='utf-8') as f:
    json.dump(all_questions, f, ensure_ascii=False, indent=4)

In [ ]:
# 定义判断检索结果是否一致的函数
from typing import Tuple, List
from langchain_core.documents import Document

def is_same_result(
        dense_res: List[Tuple[Document, float]],
        hybrid_res: List[Tuple[Document, float]]
) -> bool:
    """
    比较两个向量数据库进行密集检索的结果是否一致

    Args:
        dense_res (List[Tuple[Document, float]]): 密集检索的结果
        hybrid_res (List[Tuple[Document, float]]): 混合检索的结果

    Returns:
        bool: 检索结果是否一致
    """
    # 检查两个结果的数量是否一致
    if len(dense_res) != len(hybrid_res):
        return False

    # 检查两个结果里面的具体内容是否一样
    for d_res, h_res in zip(dense_res, hybrid_res):
        if (
            d_res[1] != h_res[1]  # 比较分数
            or d_res[0].page_content != h_res[0].page_content  # 比较文档内容
        ):
            return False

    return True

In [ ]:
# 定义查询的参数
top_k = 10
search_params = {
    'metric_type': 'L2',
    'params': {}
}

In [ ]:
# 在两个向量数据库上使用 client.search 进行检索
from tqdm import tqdm

diff_res = []
same_res_num = 0

for question in tqdm(all_questions):
    raw_dense_res = dense_vs.client.search(
        collection_name='dense_hotpotqa_500',
        data=[question['embedding']],
        anns_field='vector',  # 在 dense_hotpotqa_500 向量数据库中，密集嵌入的索引为 'vector'
        limit=top_k,
        search_params=search_params,
        output_fields=["*"]
    )
    dense_res = dense_vs._parse_documents_from_search_results(raw_dense_res)


    raw_hybrid_res = hybrid_vs.client.search(
        collection_name='hybrid_hotpotqa_500',
        data=[question['embedding']],
        anns_field='dense',
        limit=top_k,
        search_params=search_params,
        output_fields=["*"]
    )
    hybrid_res = hybrid_vs._parse_documents_from_search_results(raw_hybrid_res)

    # 比较检索结果
    if is_same_result(dense_res, hybrid_res):
        same_res_num += 1
    else:
        tmp = {
            'question': question['question'],
            'dense_res': dense_res,
            'hybrid_res': hybrid_res
        }
        diff_res.append(tmp)

        # log.info(f'{question["question"]} 检索结果不一致')
        tqdm.write(f'{question["question"]} 检索结果不一致')
# 输出结果
log.info(f'检索结果一致的问题数量：{same_res_num} / {len(all_questions)}')
log.info(f'检索结果不一致的问题数量：{len(diff_res)}')

  2%|▏         | 8/500 [00:00<00:06, 78.47it/s]

How long did the career span of the actor who starred with Mickey Rooney and Marilyn Maxwell in Off Limits? 检索结果不一致
Who was the running mate of the man about whose candidacy the 2016 book Insane Clown President: Dispatches from the 2016 Circus documents? 检索结果不一致
What relationship does Fred Gehrke have to the 23rd overall pick in the 2010 Major League Baseball Draft? 检索结果不一致


  5%|▌         | 27/500 [00:00<00:05, 80.15it/s]

Spider9 was founded in 2011 by the head of which subsidiary of Wanxiang Group? 检索结果不一致
John Stoltenberg is the managing editor of the magazine that focuses on what? 检索结果不一致
What kind of song did Alexander Grant produce for Eminem? 检索结果不一致
Is Rick Husband Amarillo International Airport or Grand Forks International Airport closer to the town it is near? 检索结果不一致
The Greatest Event in Television History aired on the channel that was founded by whom? 检索结果不一致
The Lowry Hotel is named after an artist, many of whose drawings and paintings depict what town? 检索结果不一致
ICI House is now named after the company that provides what type of item? 检索结果不一致
What professionn does Stanley Kwan and Ken Hughes have in common? 检索结果不一致
The musician who did "Khalaara" was born on what date? 检索结果不一致


  9%|▉         | 45/500 [00:00<00:05, 80.64it/s]

...

Brad Elterman is a professional photographer who addressed the rock 'n' roll lifestyle of a band that became noted for what? 检索结果不一致
Who voices the character that stars in Baseball Bugs? 检索结果不一致
Carlo Buccirosso had a role in a movie that premiered at what film festival? 检索结果不一致
Which airport is operated by the Coos Country Airport District, Southwest Oregon Regional Airport or Lawton–Fort Sill Regional Airport? 检索结果不一致
What man is linked to both Johnny Tremain and Magic Kingdom? 检索结果不一致
Why is Bangor Daily News talkin about Sawin Millett? 检索结果不一致


 97%|█████████▋| 487/500 [00:06<00:00, 82.83it/s]

Heinrich Marschner was a composer who performed in the time frame after one of the first significant composers in what school of work? 检索结果不一致
What city does Paul Clyne and David Soares have in common? 检索结果不一致
Are both Grapico and Izze artificially flavored soft drinks? 检索结果不一致
Who is from farther west, Halestorm or Audioslave? 检索结果不一致
The revue Putting It Together was devised by Stephen Sondheim and what actress best known as Miss Marple? 检索结果不一致
Who was born in Ireland and has coached the captain of MLS club Colorado Rapids? 检索结果不一致
Co-Dependents' Day is an episode of a tv show season which began when? 检索结果不一致
What is the estimated population of the city in which the suburb of Beaumont is located ? 检索结果不一致


100%|██████████| 500/500 [00:06<00:00, 80.16it/s]
2025-07-25 09:46:39.684 | INFO     | __main__:<module>:43 - 检索结果一致的问题数量：314 / 500
2025-07-25 09:46:39.684 | INFO     | __main__:<module>:44 - 检索结果不一致的问题数量：186


From what county is Steven Avery, wrongfully convicted man on which the documentary Making a Murderer is based?  检索结果不一致
